# TravelTide


## Overview
This report offers a deep dive into your customers' travel behaviors and preferences. By analyzing your extensive data, we aim to uncover what drives your travelers and how to keep them engaged. Our approach segments customers into distinct groups or "traveler tribes," allowing for the customization of rewards and incentives that feel uniquely tailored to each group. This analysis distills vast data into clear, actionable metrics that shed light on travel patterns, service preferences, and responsiveness to promotions. The ultimate goal is to enhance customer loyalty and drive business success by keeping your travelers engaged with your services.

## Approach and Data Analysis
This section outlines how customer segmentation can be applied within TravelTide to create targeted rewards and incentives. By categorizing customers based on their behaviors and preferences, we can significantly improve customer satisfaction and loyalty.

Our objective is to leverage segmentation to deliver personalized perks that boost loyalty, engagement, and overall satisfaction. We aim to offer at least five specific perks:

## Reward Offerings:

-  Exclusive Discounts
- 1 Night Free Hotel with Flight
- No Cancellation Fees
- Free Hotel Meal
- Free Checked Bag

## Data Processing and Metrics Creation
Starting with a dataset of 50,570 sessions, we distilled the information into 5,998 unique customers, using PostgreSQL to generate key metrics. This process was essential for simplifying the raw data into manageable and insightful variables.

Note: The SQL script file is in SQL folder.

## Key Metrics for Analysis
To determine the most suitable perks for each customer, we focused on the following aggregated metrics:

- Flight/Hotel/Both Preferences
- Age
- Age Group
- Conversion Rate
- Cancellation Rate
- Total Sessions
- Total Trips Booked
- Engagement (Click Efficiency)
- User Activity Level
- Average Checked Bags
- Discount Responsiveness
- Proportions of Discounts (Flights/Hotels/Both)
- Average Offers Received
- Flight Hunter Index
- Hotel Hunter Index
  
We are currently refining the calculation of the Flight Hunter Index, which involves determining the distance between geographic points while considering the Earth's shape. To achieve this, we utilized the geopy.distance library’s geodesic function.

## Packages need to be installed
- pip install scikit-learn
- pip install geopy


In [1]:
import pandas as pd
import pandas as pd
import os
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from datetime import datetime
from geopy.distance import geodesic

# custom functions 
import Support.dbSupport as dbs
import Support.calSupport as cs

## Db Connection setup

In [2]:
current_dir = os.getcwd()

In [3]:
dbs.check_tables()

['users', 'hotels', 'flights', 'sessions']

In [4]:
dbs.table_row_count()

users: 1020926 records
hotels: 1918617 records
flights: 1901038 records
sessions: 5408063 records


In [5]:
##Full_data.sql
sql_file_path_full = os.path.join(current_dir,'SQL','Full_data.sql')
#Combained query
sql_file_path_com = os.path.join(current_dir, 'SQL', 'Combained.sql')



In [6]:
CombainedData = dbs.execute_sql_file(sql_file_path_com)
fullData = dbs.execute_sql_file(sql_file_path_full)

In [7]:
CombainedData.head()

,user_id,birthdate,gender,married,has_children,home_country,home_city,age,age_group,latest_session,...,average_hotel_discount,average_flight_discount,flight_discount_proportion,hotel_discount_proportion,both_discount_proportion,discount_responsiveness,total_hotel_usd_spent,total_flight_usd_spent,total_usd_spent,hotel_hunter_index
0,23557,1958-12-08,F,True,False,usa,new york,65.0,65+,2023-07-14,...,0.175,0.150000,0.083333,0.166667,0.0,0.083333,563.0,1344.96,1907.96,0.004796
1,94883,1972-03-16,F,True,False,usa,kansas city,52.0,45-54,2023-05-28,...,0.075,0.100000,0.083333,0.166667,0.0,0.027778,230.0,5354.86,5584.86,0.000000
2,101486,1972-12-07,F,True,True,usa,tacoma,51.0,45-54,2023-07-18,...,0.000,0.075000,0.307692,0.000000,0.0,0.000000,1195.0,5994.28,7189.28,0.000000
3,101961,1980-09-14,F,True,False,usa,boston,43.0,35-44,2023-06-22,...,0.100,0.133333,0.250000,0.083333,0.0,0.011905,1052.0,1929.20,2981.20,0.000000
4,106907,1978-11-17,F,True,True,usa,miami,45.0,45-54,2023-07-27,...,0.200,0.150000,0.071429,0.071429,0.0,0.017857,1185.0,27969.63,29154.63,0.001623


In [8]:
CombainedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   user_id                     5998 non-null   int64  
 1   birthdate                   5998 non-null   object 
 2   gender                      5998 non-null   object 
 3   married                     5998 non-null   bool   
 4   has_children                5998 non-null   bool   
 5   home_country                5998 non-null   object 
 6   home_city                   5998 non-null   object 
 7   age                         5998 non-null   float64
 8   age_group                   5998 non-null   object 
 9   latest_session              5998 non-null   object 
 10  total_trips                 5998 non-null   int64  
 11  total_cancellations         5998 non-null   int64  
 12  total_sessions              5998 non-null   int64  
 13  total_cancellation_rate     5998 

In [9]:
fullData.head()

,user_id,trip_id,fd_amount,base_fare_usd,home_airport_lat,home_airport_lon,destination_airport_lat,destination_airport_lon
0,23557,23557-3354bee182614ec8afc585d907234203,0.0,623.25,40.777,-73.872,53.667,-113.467
1,23557,23557-753d651e2b6a493080bcf880b1ce7cba,0.0,0.00,40.777,-73.872,NaN,NaN
2,23557,23557-3354bee182614ec8afc585d907234203,0.0,623.25,40.777,-73.872,53.667,-113.467
3,23557,23557-0c179482e53242b1b28d90c33dfa6050,0.0,0.00,40.777,-73.872,NaN,NaN
4,23557,,0.0,0.00,40.777,-73.872,NaN,NaN


In [10]:
print(fullData[['home_airport_lat', 'home_airport_lon', 'destination_airport_lat', 'destination_airport_lon']].head())


   home_airport_lat  home_airport_lon  destination_airport_lat  \
0            40.777           -73.872                   53.667   
1            40.777           -73.872                      NaN   
2            40.777           -73.872                   53.667   
3            40.777           -73.872                      NaN   
4            40.777           -73.872                      NaN   

   destination_airport_lon  
0                 -113.467  
1                      NaN  
2                 -113.467  
3                      NaN  
4                      NaN  


In [11]:
# Calculate distance between home airport and destination airport
fullData['distance'] = fullData.apply(
    lambda row: geodesic(
        (row['home_airport_lat'], row['home_airport_lon']),
        (row['destination_airport_lat'], row['destination_airport_lon'])
    ).kilometers if not pd.isna(row['home_airport_lat']) and not pd.isna(row['home_airport_lon']) 
                     and not pd.isna(row['destination_airport_lat']) and not pd.isna(row['destination_airport_lon'])
                     else 0,
    axis=1
)

In [12]:
fullData.head()

,user_id,trip_id,fd_amount,base_fare_usd,home_airport_lat,home_airport_lon,destination_airport_lat,destination_airport_lon,distance
0,23557,23557-3354bee182614ec8afc585d907234203,0.0,623.25,40.777,-73.872,53.667,-113.467,3268.232887
1,23557,23557-753d651e2b6a493080bcf880b1ce7cba,0.0,0.00,40.777,-73.872,NaN,NaN,0.000000
2,23557,23557-3354bee182614ec8afc585d907234203,0.0,623.25,40.777,-73.872,53.667,-113.467,3268.232887
3,23557,23557-0c179482e53242b1b28d90c33dfa6050,0.0,0.00,40.777,-73.872,NaN,NaN,0.000000
4,23557,,0.0,0.00,40.777,-73.872,NaN,NaN,0.000000


In [13]:
# Group data by 'user_id' and calculate the total discount and total distance for each user
grouped_by_user_id = fullData.groupby('user_id').agg(
    total_discount_amount=pd.NamedAgg(column='fd_amount', aggfunc='sum'),
    total_distance=pd.NamedAgg(column='distance', aggfunc='sum')
).reset_index()

In [14]:
# Calculate ADS/KM for each user
grouped_by_user_id['ads_per_km'] = grouped_by_user_id.apply(
    lambda row: row['total_discount_amount'] / row['total_distance'] if row['total_distance'] != 0 else 0,
    axis=1
)

In [15]:
# Min-Max Scaling of ADS/KM values to bring them within [0, 1] range
min_ads_per_km = grouped_by_user_id['ads_per_km'].min()
max_ads_per_km = grouped_by_user_id['ads_per_km'].max()
grouped_by_user_id['scaled_ads_per_km'] = (grouped_by_user_id['ads_per_km'] - min_ads_per_km) / (max_ads_per_km - min_ads_per_km)


In [16]:
# Check the total tally with 5998 records
grouped_by_user_id

,user_id,total_discount_amount,total_distance,ads_per_km,scaled_ads_per_km
0,23557,0.15,7104.444963,0.000021,0.001122
1,94883,0.10,15125.233276,0.000007,0.000351
2,101486,0.30,10304.420174,0.000029,0.001548
3,101961,0.40,10824.484503,0.000037,0.001964
4,106907,0.15,28847.623626,0.000005,0.000276
...,...,...,...,...,...
5993,792549,0.15,5761.414327,0.000026,0.001384
5994,796032,0.10,17204.909863,0.000006,0.000309
5995,801660,0.50,2168.360039,0.000231,0.012258
5996,811077,0.15,3192.139217,0.000047,0.002498
